In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [6]:
df=pd.read_csv("C:/Users/sai/Downloads/UahZCQ.csv")
df=df.sample(frac=1,random_state=4).reset_index(drop=True)

In [7]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,1,100,66,29,196,32.0,0.444,42,0
1,9,57,80,37,0,32.8,0.096,41,0
2,0,100,70,26,50,30.8,0.597,21,0
3,1,119,88,41,170,45.3,0.507,26,0
4,2,102,86,36,120,45.5,0.127,23,1


In [8]:
df.isnull().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
Pregnancies                 768 non-null int64
Glucose                     768 non-null int64
BloodPressure               768 non-null int64
SkinThickness               768 non-null int64
Insulin                     768 non-null int64
BMI                         768 non-null float64
DiabetesPedigreeFunction    768 non-null float64
Age                         768 non-null int64
Outcome                     768 non-null int64
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [10]:
df.shape

(768, 9)

In [11]:
df.ndim

2

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
y=df['Outcome']
x=df.drop('Outcome',axis=1)

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=42)

In [14]:
x_train.shape

(537, 8)

In [15]:
x_test.shape

(231, 8)

In [16]:
from sklearn.preprocessing import StandardScaler
ss=StandardScaler()

x_trains=ss.fit_transform(x_train)
x_tests=ss.transform(x_test)

In [19]:
from sklearn.neighbors import KNeighborsClassifier
knn=KNeighborsClassifier()
knn.fit(x_trains,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [21]:
from sklearn.metrics import accuracy_score,confusion_matrix,roc_auc_score,roc_curve
y_train_pred=knn.predict(x_trains)
y_train_prob=knn.predict_proba(x_trains)[:,1]
print('train - confusion matrix : ','\n',confusion_matrix(y_train,y_train_pred))
print('train - accuracy score : ','\n', accuracy_score(y_train,y_train_pred))
print('train - AUC : ', roc_auc_score(y_train,y_train_prob))

train - confusion matrix :  
 [[308  41]
 [ 57 131]]
train - accuracy score :  
 0.8175046554934823
train - AUC :  0.8982045967201122


In [22]:
y_test_pred=knn.predict(x_tests)
y_test_prob=knn.predict_proba(x_tests)[:,1]
print('test - confusion matrix : ','\n',confusion_matrix(y_test,y_test_pred))
print('test - accuracy score : ','\n', accuracy_score(y_test,y_test_pred))
print('test - AUC : ', roc_auc_score(y_test,y_test_prob))

test - confusion matrix :  
 [[133  18]
 [ 42  38]]
test - accuracy score :  
 0.7402597402597403
test - AUC :  0.764114238410596


# hyper parameter tuning

In [26]:
xs=ss.fit_transform(x)

In [30]:
from scipy.stats import randint as sp_randint
from sklearn.model_selection import RandomizedSearchCV
param_dist={'n_neighbors':sp_randint(1,15),
           'p':sp_randint(1,6)}
r_search=RandomizedSearchCV(knn,param_distributions=param_dist,cv=3,n_iter=100,scoring='roc_auc',random_state=4,n_jobs=-1)


print(r_search.fit(xs,y))
print(r_search.best_params_)

RandomizedSearchCV(cv=3, error_score='raise-deprecating',
                   estimator=KNeighborsClassifier(algorithm='auto',
                                                  leaf_size=30,
                                                  metric='minkowski',
                                                  metric_params=None,
                                                  n_jobs=None, n_neighbors=5,
                                                  p=2, weights='uniform'),
                   iid='warn', n_iter=100, n_jobs=-1,
                   param_distributions={'n_neighbors': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000025B65D5F108>,
                                        'p': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000025B65D5F388>},
                   pre_dispatch='2*n_jobs', random_state=4, refit=True,
                   return_train_score=False, scoring='roc_auc', verbose=0)
{'n_neighbors': 14, 'p': 1}


In [28]:
from scipy.stats import randint as sp_randint
from sklearn.model_selection import RandomizedSearchCV
param_dist={'n_neighbors':sp_randint(1,20),
           'p':sp_randint(1,6)}
r_search=RandomizedSearchCV(knn,param_distributions=param_dist,cv=3,n_iter=100,scoring='roc_auc',random_state=4,n_jobs=-1)


print(r_search.fit(xs,y))
print(r_search.best_params_)

RandomizedSearchCV(cv=3, error_score='raise-deprecating',
                   estimator=KNeighborsClassifier(algorithm='auto',
                                                  leaf_size=30,
                                                  metric='minkowski',
                                                  metric_params=None,
                                                  n_jobs=None, n_neighbors=5,
                                                  p=2, weights='uniform'),
                   iid='warn', n_iter=100, n_jobs=-1,
                   param_distributions={'n_neighbors': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000025B65C53748>,
                                        'p': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000025B65D292C8>},
                   pre_dispatch='2*n_jobs', random_state=4, refit=True,
                   return_train_score=False, scoring='roc_auc', verbose=0)
{'n_neighbors': 16, 'p': 2}


In [33]:
from scipy.stats import randint as sp_randint
from sklearn.model_selection import RandomizedSearchCV
param_dist={'n_neighbors':sp_randint(1,25),
           'p':sp_randint(1,6)}
r_search=RandomizedSearchCV(knn,param_distributions=param_dist,cv=3,n_iter=100,scoring='roc_auc',random_state=4,n_jobs=-1)


print(r_search.fit(xs,y))
print(r_search.best_params_)

RandomizedSearchCV(cv=3, error_score='raise-deprecating',
                   estimator=KNeighborsClassifier(algorithm='auto',
                                                  leaf_size=30,
                                                  metric='minkowski',
                                                  metric_params=None,
                                                  n_jobs=None, n_neighbors=14,
                                                  p=1, weights='uniform'),
                   iid='warn', n_iter=100, n_jobs=-1,
                   param_distributions={'n_neighbors': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000025B65D717C8>,
                                        'p': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000025B65D71C48>},
                   pre_dispatch='2*n_jobs', random_state=4, refit=True,
                   return_train_score=False, scoring='roc_auc', verbose=0)
{'n_neighbors': 23, 'p': 1}


In [34]:
knn=KNeighborsClassifier(**r_search.best_params_)
knn.fit(x_trains,y_train)
y_train_pred=knn.predict(x_trains)
y_train_prob=knn.predict_proba(x_trains)[:,1]
print('train - confusion matrix : ','\n',confusion_matrix(y_train,y_train_pred))
print('train - accuracy score : ','\n', accuracy_score(y_train,y_train_pred))
print('train - AUC : ', roc_auc_score(y_train,y_train_prob))

train - confusion matrix :  
 [[321  28]
 [ 96  92]]
train - accuracy score :  
 0.7690875232774674
train - AUC :  0.8581433274401025


In [36]:
# for more number of n_neighbors the accuracy increases more
y_test_pred=knn.predict(x_tests)
y_test_prob=knn.predict_proba(x_tests)[:,1]  
print('test - confusion matrix : ','\n',confusion_matrix(y_test,y_test_pred))
print('test - accuracy score : ','\n', accuracy_score(y_test,y_test_pred))
print('test - AUC : ', roc_auc_score(y_test,y_test_prob))

test - confusion matrix :  
 [[136  15]
 [ 47  33]]
test - accuracy score :  
 0.7316017316017316
test - AUC :  0.8028973509933776
